<a href="https://colab.research.google.com/github/Nemczek/pycheckio_database/blob/main/pyCheckio_class_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project pyCheckio

## Setting up

In [1]:
# importing libraries
import requests
import pandas as pd
import sqlalchemy as db

In [65]:
# static variables
BASE_URL = 'https://py.checkio.org/api/group-details/'
GROUP_PROGRESS_API_BASE = 'https://py.checkio.org/api/group-progress/'
GROUP_ACTIVITY_API_BASE = 'https://py.checkio.org/api/group-activity/'
TOKEN = None

URL_WITH_TOKEN = BASE_URL + TOKEN
PROGRESS_API_WITH_TOKEN = GROUP_PROGRESS_API_BASE + TOKEN
ACTIVITY_API_WITH_TOKEN = GROUP_ACTIVITY_API_BASE + TOKEN

In [27]:
# listing all classes
requests.get(URL_WITH_TOKEN).json()['objects']

[{'name': '2021-2022 Jezyki programowania 1',
  'slug': 'jezyki-programowania-1',
  'default_language': 'en',
  'description_text': '',
  'created_at': '2021-10-06',
  'membership_admission': 'Anyone but the approval is required',
  'is_auto_following_enabled': True,
  'is_email_review_activated': True,
  'is_group_default_filter': False,
  'is_full_access': False,
  'is_leaderboard_enabled': True,
  'is_monthly_leaderboard_default': False,
  'is_revisions_enabled': True,
  'is_member_profiles_visible_for_site': False,
  'is_member_solutions_visible_for_site': False,
  'is_site_solutions_visible_for_members': True,
  'members_count': 25,
  'last_activity': '2023-03-24 12:41',
  'course': 'CheckiO [Easy] (no points for votes)',
  'is_current_group': False},
 {'name': 'stara Języki programowania 1 grupa 2',
  'slug': 'jezyki-programowania-1-grupa-2',
  'default_language': 'en',
  'description_text': '',
  'created_at': '2021-02-26',
  'membership_admission': 'Only by invitation via email

In [17]:
def get_slug(url):
  """
  Returns slag parameter of our current class

  Parameters:
  url (str): URL to our class details

  Returns:
  slug (str): the slug parameter 
  """
  slug = requests.get(url).json()['objects'][4]['slug'] # 4 is the index of our class
  return slug
print(get_slug(URL_WITH_TOKEN))

michal-wojcik-2022-2023


In [28]:
print(ACTIVITY_API_WITH_TOKEN)

https://py.checkio.org/api/group-activity/?token=277869ee829d44609019622889900651


## Activity API

In [ ]:
# Using Group Progress API
class_slug = get_slug(URL_WITH_TOKEN)
activity_url_with_slug = f"{ACTIVITY_API_WITH_TOKEN}&slug={class_slug}"

resp = requests.get(url_with_slug).json()['objects']
#requests.get(url_with_slug).json()['objects'][2]['data']
resp[2]

## Progress API

In [ ]:
progress_url_with_slug = f"{PROGRESS_API_WITH_TOKEN}&slug={class_slug}"
progress_data = requests.get(progress_url_with_slug).json()['objects']

In [73]:
progress_data[0]['data']

usernames = []
for i in progress_data[0]['data']:
  if i['username'] not in usernames:
    usernames.append(i['username'])
usernames

['karol2202',
 '117374',
 'Antoni_Wojcik',
 'Karolina_Zadura',
 '126083',
 '119587',
 'Mariaa_Kuczynskaa',
 '126212',
 'Agnieszka_Dziudzi',
 'HubiJohn98',
 'Kacper_Kaszuba',
 '117394',
 '117370',
 'Mateusz_Wryszcz',
 'wpzborowski',
 'natalia.rogosch',
 'JakubKania',
 '112777',
 '117357',
 '126084',
 'Jakub_Niedziolka',
 'Joanna_Szalkowska',
 'Rafal.Keszycki',
 'Tomasz_Radosz',
 'Wiktor_Mikolajczak',
 'Szymon_Gieszcz',
 '115141',
 'Gabriela_Gardas',
 'Aleksandra_Niewiadomska',
 'Karolina.Kwietniewska',
 'MichalRudolf',
 '117422',
 '112094',
 'ania.barton',
 'Studint',
 '117359',
 '117375',
 'przemek.wieczorek',
 'KacperStelmaszyk',
 '117369',
 '126086',
 'Widmo1',
 'Widmo2',
 'Widmo3',
 '115128',
 '126089',
 'AlicjaKraska',
 'Mariia_Salganik',
 'monika.sadkowska']